In [1]:
!pip install requests beautifulsoup4 tqdm selenium


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import sys

# --- 1. 설정: 님이 찾아주신 정확한 CSS 선택자 ---
BASE_URL = "https://nightcityshards.net"
OUTPUT_FILE = "cyberpunk_shards.txt"

POPUP_BUTTON_XPATH = "//button[text()='Okay']"
CATEGORY_HEADER_CSS = "h2.section_header"     # (성공) 카테고리 헤더
INDIVIDUAL_SHARD_CSS = "h3.titleBox"        # (성공) 클릭할 개별 샤드 (왼쪽 패널)

# ⭐️ (수정) 님이 찾아주신 '진짜' 본문 ID
SHARD_BODY_CSS = "section#textBody"           #

print("셀레니움 웹 드라이버를 준비합니다...")
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10) 
print(f"접속 시작: {BASE_URL}")

try:
    driver.get(BASE_URL)
    
    # --- 2. 팝업 "Okay" 버튼 클릭 ---
    print("안내 창(pop-up)의 'Okay' 버튼을 기다리는 중...")
    wait.until(EC.element_to_be_clickable((By.XPATH, POPUP_BUTTON_XPATH))).click()
    print("'Okay' 버튼 클릭 완료.")
    
    # --- 3. 모든 '카테고리 헤더' 클릭하여 확장 ---
    print(f"'{CATEGORY_HEADER_CSS}' 선택자로 카테고리 메뉴를 찾습니다...")
    headers = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, CATEGORY_HEADER_CSS))
    )
    print(f"총 {len(headers)}개의 카테고리 발견. 순서대로 클릭합니다.")
    
    for header in headers:
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", header)
            header.click()
            time.sleep(0.1) 
        except Exception:
            pass
            
    print("모든 카테고리 확장 완료.")

    # --- 4. 모든 '개별 샤드' 링크 수집 ---
    shard_links = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, INDIVIDUAL_SHARD_CSS))
    )
    total_shards = len(shard_links)
    print(f"총 {total_shards}개의 개별 샤드를 찾았습니다. 스크래핑을 시작합니다.")

    # --- 5. (핵심) '개별 샤드'를 하나씩 클릭하며 우측 내용 스크래핑 ---
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        
        for i in tqdm(range(total_shards), desc="샤드 스크래핑 중"):
            title = "제목 없음"
            body = "내용 없음"
            
            try:
                # (1) 루프마다 '개별 샤드' 목록을 새로고침 (오류 방지)
                all_links = driver.find_elements(By.CSS_SELECTOR, INDIVIDUAL_SHARD_CSS)
                link_to_click = all_links[i]
                
                # (2) ⭐️ (수정) 님이 관찰한 대로, '제목'은 클릭하는 대상에서 미리 가져옴
                #
                title = link_to_click.text 
                
                # (3) 왼쪽 패널에서 샤드 링크 클릭
                driver.execute_script("arguments[0].scrollIntoView(true);", link_to_click)
                link_to_click.click()
                
                # (4) ⭐️ (수정) 님이 찾아주신 '진짜' 본문 ID (section#textBody)가 나타날 때까지 대기
                #
                body_element = wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, SHARD_BODY_CSS))
                )
                
                # (5) 우측 패널에서 본문 텍스트 스크래핑
                body = body_element.text
                
                # (6) 파일에 저장
                f.write(f"[샤드 제목: {title.strip()}]\n")
                f.write(f"{body.strip()}\n")
                f.write("----\n\n")

            except Exception as e:
                # (7) 오류 발생 시, 어떤 샤드에서 왜 오류가 났는지 출력
                print(f"\n[오류] 샤드 #{i} ('{title}') 처리 중 문제 발생:")
                print(f"  - 오류 타입: {type(e).__name__}")
                
    print(f"\n🎉 스크래핑 완료! {OUTPUT_FILE} 파일에 저장되었습니다.")

except Exception as e:
    print(f"스크립트 실행 중 치명적인 오류 발생: {e}", file=sys.stderr)
finally:
    if 'driver' in locals():
        driver.quit()

셀레니움 웹 드라이버를 준비합니다...
접속 시작: https://nightcityshards.net
안내 창(pop-up)의 'Okay' 버튼을 기다리는 중...
'Okay' 버튼 클릭 완료.
'h2.section_header' 선택자로 카테고리 메뉴를 찾습니다...
총 13개의 카테고리 발견. 순서대로 클릭합니다.
모든 카테고리 확장 완료.
총 1228개의 개별 샤드를 찾았습니다. 스크래핑을 시작합니다.


샤드 스크래핑 중: 100%|██████████| 1228/1228 [01:28<00:00, 13.91it/s]



🎉 스크래핑 완료! cyberpunk_shards.txt 파일에 저장되었습니다.
